# Binary Classification Model for Interview

*Dustin Bracy, Paul Huggins, Grace Lang & Branum Stephens* 

## Analysis Objective: 
The purpose of this analysis is to build and tune a model best fit for classifying a binary field of 'y'. During an interview set by the hiring manager we were given autonomized data that will be used in the analysis. The data fields and descriptions are unknown, so we will rely on our modeling techniques and evaluation to determine the best fit model for a classification prediction. 

## Reading in the data: 

In [1]:
!pip install bayesian-optimization
!pip install hpsklearn
!pip install xgboost
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot
from numpy import sort
from sklearn.feature_selection import SelectFromModel

In [2]:
#I could not push this file to git. So you'll have to download yourself and save within your local repository
#data sourced: https://smu.box.com/s/k9x192jxm39enjw2wx8ouw2kopx33l32
#df = pd.read_csv("../Unit14_CaseStudy/final_project.csv")

# read in data
df = pd.read_csv("C:/Users/Paulh/Downloads/final_project.csv")

## EDA of Dataset:

In [3]:
#Running Pandas Profile Report
# profile = ProfileReport(df, title='Pandas Profiling Report')

In [4]:
#profile.to_widgets()

In [5]:
df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x41,x42,x43,x44,x45,x46,x47,x48,x49,y
0,-0.166563,-3.961588,4.621113,2.481908,-1.800135,0.804684,6.718751,-14.789997,-1.040673,-4.204950,...,-1.497117,5.414063,-2.325655,1.674827,-0.264332,60.781427,-7.689696,0.151589,-8.040166,0
1,-0.149894,-0.585676,27.839856,4.152333,6.426802,-2.426943,40.477058,-6.725709,0.896421,0.330165,...,36.292790,4.490915,0.762561,6.526662,1.007927,15.805696,-4.896678,-0.320283,16.719974,0
2,-0.321707,-1.429819,12.251561,6.586874,-5.304647,-11.311090,17.812850,11.060572,5.325880,-2.632984,...,-0.368491,9.088864,-0.689886,-2.731118,0.754200,30.856417,-7.428573,-2.090804,-7.869421,0
3,-0.245594,5.076677,-24.149632,3.637307,6.505811,2.290224,-35.111751,-18.913592,-0.337041,-5.568076,...,15.691546,-7.467775,2.940789,-6.424112,0.419776,-72.424569,5.361375,1.806070,-7.670847,0
4,-0.273366,0.306326,-11.352593,1.676758,2.928441,-0.616824,-16.505817,27.532281,1.199715,-4.309105,...,-13.911297,-5.229937,1.783928,3.957801,-0.096988,-14.085435,-0.208351,-0.894942,15.724742,1


In [6]:
df.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x41,x42,x43,x44,x45,x46,x47,x48,x49,y
count,159974.000000,159975.000000,159962.000000,159963.000000,159974.000000,159963.000000,159974.000000,159973.000000,159979.000000,159970.000000,...,159960.000000,159974.000000,159963.000000,159960.000000,159971.000000,159969.000000,159963.000000,159968.000000,159968.000000,160000.000000
mean,-0.001028,0.001358,-1.150145,-0.024637,-0.000549,0.013582,-1.670670,-7.692795,-0.030540,0.005462,...,6.701076,-1.833820,-0.002091,-0.006250,0.000885,-12.755395,0.028622,-0.000224,-0.674224,0.401231
std,0.371137,6.340632,13.273480,8.065032,6.382293,7.670076,19.298665,30.542264,8.901185,6.355040,...,18.680196,5.110705,1.534952,4.164595,0.396621,36.608641,4.788157,1.935501,15.036738,0.490149
min,-1.592635,-26.278302,-59.394048,-35.476594,-28.467536,-33.822988,-86.354483,-181.506976,-37.691045,-27.980659,...,-82.167224,-27.933750,-6.876234,-17.983487,-1.753221,-201.826828,-21.086333,-8.490155,-65.791191,0.000000
25%,-0.251641,-4.260973,-10.166536,-5.454438,-4.313118,-5.148130,-14.780146,-27.324771,-6.031058,-4.260619,...,-5.804080,-5.162869,-1.039677,-2.812055,-0.266518,-36.428329,-3.216016,-1.320800,-10.931753,0.000000
50%,-0.002047,0.004813,-1.340932,-0.031408,0.000857,0.014118,-1.948594,-6.956789,-0.016840,0.006045,...,6.840110,-1.923754,-0.004385,-0.010484,0.001645,-12.982497,0.035865,-0.011993,-0.574410,0.000000
75%,0.248532,4.284220,7.871676,5.445179,4.306660,5.190749,11.446931,12.217071,5.972349,4.305734,...,19.266367,1.453507,1.033275,2.783274,0.269049,11.445443,3.268028,1.317703,9.651072,1.000000
max,1.600849,27.988178,63.545653,38.906025,26.247812,35.550110,92.390605,149.150634,39.049831,27.377842,...,100.050432,22.668041,6.680922,19.069759,1.669205,150.859415,20.836854,8.226552,66.877604,1.000000


In [7]:
# data types and null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160000 entries, 0 to 159999
Data columns (total 51 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   x0      159974 non-null  float64
 1   x1      159975 non-null  float64
 2   x2      159962 non-null  float64
 3   x3      159963 non-null  float64
 4   x4      159974 non-null  float64
 5   x5      159963 non-null  float64
 6   x6      159974 non-null  float64
 7   x7      159973 non-null  float64
 8   x8      159979 non-null  float64
 9   x9      159970 non-null  float64
 10  x10     159957 non-null  float64
 11  x11     159970 non-null  float64
 12  x12     159964 non-null  float64
 13  x13     159969 non-null  float64
 14  x14     159966 non-null  float64
 15  x15     159965 non-null  float64
 16  x16     159974 non-null  float64
 17  x17     159973 non-null  float64
 18  x18     159960 non-null  float64
 19  x19     159965 non-null  float64
 20  x20     159962 non-null  float64
 21  x21     15

In [8]:
df.isna().sum()

x0     26
x1     25
x2     38
x3     37
x4     26
x5     37
x6     26
x7     27
x8     21
x9     30
x10    43
x11    30
x12    36
x13    31
x14    34
x15    35
x16    26
x17    27
x18    40
x19    35
x20    38
x21    29
x22    27
x23    47
x24    28
x25    22
x26    36
x27    30
x28    35
x29    30
x30    30
x31    39
x32    31
x33    41
x34    41
x35    30
x36    27
x37    23
x38    31
x39    23
x40    36
x41    40
x42    26
x43    37
x44    40
x45    29
x46    31
x47    37
x48    32
x49    32
y       0
dtype: int64

In [9]:
# extract numeric columns
num_cols = df.columns[df.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
df[num_cols]

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x41,x42,x43,x44,x45,x46,x47,x48,x49,y
0,-0.166563,-3.961588,4.621113,2.481908,-1.800135,0.804684,6.718751,-14.789997,-1.040673,-4.204950,...,-1.497117,5.414063,-2.325655,1.674827,-0.264332,60.781427,-7.689696,0.151589,-8.040166,0
1,-0.149894,-0.585676,27.839856,4.152333,6.426802,-2.426943,40.477058,-6.725709,0.896421,0.330165,...,36.292790,4.490915,0.762561,6.526662,1.007927,15.805696,-4.896678,-0.320283,16.719974,0
2,-0.321707,-1.429819,12.251561,6.586874,-5.304647,-11.311090,17.812850,11.060572,5.325880,-2.632984,...,-0.368491,9.088864,-0.689886,-2.731118,0.754200,30.856417,-7.428573,-2.090804,-7.869421,0
3,-0.245594,5.076677,-24.149632,3.637307,6.505811,2.290224,-35.111751,-18.913592,-0.337041,-5.568076,...,15.691546,-7.467775,2.940789,-6.424112,0.419776,-72.424569,5.361375,1.806070,-7.670847,0
4,-0.273366,0.306326,-11.352593,1.676758,2.928441,-0.616824,-16.505817,27.532281,1.199715,-4.309105,...,-13.911297,-5.229937,1.783928,3.957801,-0.096988,-14.085435,-0.208351,-0.894942,15.724742,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,-0.487024,-4.270269,0.417395,-1.992423,1.757552,-1.167819,0.606860,41.084463,-1.923188,-2.374213,...,-9.390451,8.096802,-0.875131,-1.413787,-0.363968,15.339392,4.364205,-3.831489,28.389858,1
159996,0.825477,4.804368,22.161535,11.371303,1.715901,6.990759,32.221207,-12.278038,-3.861086,6.715126,...,12.803189,0.841446,-0.682177,-5.047677,-0.017898,0.780130,6.387266,-1.374742,-1.623952,0
159997,-0.802489,5.362696,7.243419,-7.496074,2.295250,-2.756067,10.531388,42.515821,1.420984,6.788916,...,-0.346570,-0.144098,0.738298,7.241041,0.215347,-12.155249,3.265263,1.230963,3.335471,1
159998,0.339237,7.609895,5.368414,-2.825481,4.046102,15.322603,7.805271,-10.233054,2.609986,4.251127,...,-0.307656,-0.601145,-3.443112,0.549931,0.206728,5.081980,1.701462,-0.279619,-1.986424,0


In [10]:
# normalize numeric columns
num_cols = df.columns[df.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [11]:
[x for x in df if type(df[x][0]) != np.float64]

['x24', 'x29', 'x30', 'x32', 'x37']

In [12]:
[print(x,type(x)) for x in df if type(df[x][0]) != np.float64]

x24 <class 'str'>
x29 <class 'str'>
x30 <class 'str'>
x32 <class 'str'>
x37 <class 'str'>


[None, None, None, None, None]

In [13]:
cat_vars = [col for col in df if type(df[col][0]) != np.float64]

df[cat_vars]

,x24,x29,x30,x32,x37
0,euorpe,July,tuesday,0.0%,$1313.96
1,asia,Aug,wednesday,-0.02%,$1962.78
2,asia,July,wednesday,-0.01%,$430.47
3,asia,July,wednesday,0.01%,$-2366.29
4,asia,July,tuesday,0.01%,$-620.66
...,...,...,...,...,...
159995,asia,Aug,wednesday,0.0%,$-891.96
159996,asia,May,wednesday,-0.01%,$1588.65
159997,asia,Jun,wednesday,-0.0%,$687.46
159998,asia,May,wednesday,-0.02%,$439.21


In [14]:
month_fix = {
    'July':7, 
    'Aug':8, 
    'Jun':6, 
    'May':5, 
    'sept.':9, 
    'Apr':4, 
    'Nov':11, 
    'Oct':10, 
    'Mar':3,
    'Feb':2, 
    'Dev':12, 
    'January':1
}

week_fix = {'tuesday':2, 'wednesday':3, 'thurday':4, 'monday':1, 'friday':5}

df['x29'] = df['x29'].apply(lambda x: month_fix.get(x))
df['x30'] = df['x30'].apply(lambda x: week_fix.get(x))

In [15]:
def strip_special(x):
    if type(x) != str:
        return x
    clean = x.strip('$')
    if clean.endswith('%'):
        clean = x.strip('%')
        clean = np.float64(clean) * .01
    return np.float64(clean)

In [16]:
df['x32'] = df['x32'].apply(strip_special)
df['x37'] = df['x37'].apply(strip_special)


In [17]:
# use pd.concat to join the new columns with original dataframe
df = pd.concat([df,pd.get_dummies(df['x24'], prefix='x24')],axis=1)

# drop the original 'x24' column
df.drop(['x24'],axis=1, inplace=True)

In [18]:
df.dropna(inplace=True)
X = df.drop(columns=['y'])
y = df['y']
assert len(X) == len(y), 'X/Y are different lengths'

# encode string class values as integers
label_encoded_y = LabelEncoder().fit_transform(y)

In [19]:
label_encoded_y

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [20]:
# split data into train and test sets
seed = 123
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, label_encoded_y, test_size=test_size, random_state=seed)

In [25]:
# baseline xgboost to build upon

# fit model to training data
model = XGBClassifier(random_state = 123,verbosity=0, use_label_encoder=False)
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("XGBoost Baseline Model Accuracy: %.2f%%" % (accuracy * 100.0))

XGBoost Baseline Model Accuracy: 92.54%


In [ ]:
# warning -- long run time --
# the below parameter search grid was run to determine optimal parameters.
# warning -- long run time --

model = XGBClassifier()
param_grid = {
    'booster' : ['gbtree', 'gblinear', 'dart'],
	"grow_policy" : ["lossguide", 'depthwise'],
    "max_depth" : range(1,12,1),
    "n_estimators" : range(50,500,50),
	"colsample_bytree" : range(0.1,1,0.1),
    "eta" : range(0.1, 1, 0.05),
    "gamma" : range(0,1,0.5),
}
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold, verbosity=0, use_label_encoder=False)
grid_result = grid_search.fit(X, label_encoded_y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# tuned xgboost based on grid search from above

# fit model to training data
model = XGBClassifier(booster = "gbtree", eta = 0.5, gamma = 0, max_depth = 10, colsample_bytree = 0.9, grow_policy = "lossguide", max_leaves = 0, random_state = 123, verbosity=0, use_label_encoder=False)
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("XGBoost Tuned Model Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# build xgboost model using feature importance
# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_)
for thresh in thresholds:
	# select features using threshold
	selection = SelectFromModel(model, threshold=thresh, prefit=True)
	select_X_train = selection.transform(X_train)
	# train model
	selection_model = XGBClassifier(booster = "gbtree", eta = 0.5, gamma = 0, max_depth = 10, colsample_bytree = 0.9, grow_policy = "lossguide", max_leaves = 0, random_state = 123, verbosity=0, use_label_encoder=False)
	selection_model.fit(select_X_train, y_train)
	# eval model
	select_X_test = selection.transform(X_test)
	predictions = selection_model.predict(select_X_test)
	accuracy = accuracy_score(y_test, predictions)
	print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

In [ ]:
# feature importance
for col,score in zip(X_train.columns,model.feature_importances_):
    print(col,'Score:', score)

Our baseline xgboost model achieved an accoracy of 92.54%. 

A tuned xgboost model using an extensive search parameter grid achieved an accuracy of 93.86%. While this is improvement, the grid search is very computationally expensive. 

Uing a Feature Selection approach on top of the optimal parmeters discovered in the grid search, we can achieve an accuracy of 94.63% using the top 16 features based in their feature importance instead of the full 52 columns. Not only does this achieve highest accuracy, it also reduces processing time allowing for resources to be spent in other areas of the team.